In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
import tensorflow as tf
labels = []
imgs = []
for dirname, _, filenames in os.walk('/kaggle/input/rgb-arabic-alphabets-sign-language-dataset/RGB ArSL dataset'):
    for filename in filenames:
        labels.append(dirname.split('/')[-1])
        img = cv2.imread(os.path.join(dirname,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(224,224))
        img = img /255.0
        img= tf.convert_to_tensor(img,dtype=tf.float32)
        imgs.append(img)
imgs = tf.stack(imgs)
labels = np.array(labels)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Premature end of JPEG file


In [9]:
labels

array(['Zain', 'Zain', 'Zain', ..., 'Laa', 'Laa', 'Laa'], dtype='<U11')

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(
    np.array(imgs), labels, test_size=0.2, random_state=42, shuffle=True
)

In [11]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(31)
])


In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
early_stopping = EarlyStopping(
    monitor='val_loss',   
    patience=3,          
    restore_best_weights=True 
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test,y_test),
    epochs=10,
    batch_size=64,
    callbacks = [early_stopping]
)

Epoch 1/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 43s 308ms/step - accuracy: 0.0311 - loss: 3.4857 - val_accuracy: 0.0573 - val_loss: 3.4270
Epoch 2/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 21s 207ms/step - accuracy: 0.0454 - loss: 3.4288 - val_accuracy: 0.0573 - val_loss: 3.4163
Epoch 3/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 22s 218ms/step - accuracy: 0.0460 - loss: 3.4157 - val_accuracy: 0.0458 - val_loss: 3.4023
Epoch 4/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 21s 214ms/step - accuracy: 0.0496 - loss: 3.3965 - val_accuracy: 0.0681 - val_loss: 3.3781
Epoch 5/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 21s 209ms/step - accuracy: 0.0510 - loss: 3.3736 - val_accuracy: 0.0846 - val_loss: 3.3544
Epoch 6/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 21s 210ms/step - accuracy: 0.0614 - loss: 3.3571 - val_accuracy: 0.0763 - val_loss: 3.3332
Epoch 7/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 21s 212ms/step - accuracy: 0.0654 - loss: 3.3293 - val_accuracy: 0.0865 - val_loss: 3.3089
Epoch 8/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 21s 212ms/step - accuracy: 0.0616 - loss: 3.3139 - val_accu

In [14]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test,y_test),
    epochs=300,
    batch_size=128,
    callbacks = [early_stopping]
)

Epoch 1/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 27s 488ms/step - accuracy: 0.0667 - loss: 3.2988 - val_accuracy: 0.0827 - val_loss: 3.2749
Epoch 2/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 22s 437ms/step - accuracy: 0.0645 - loss: 3.2772 - val_accuracy: 0.0814 - val_loss: 3.2665
Epoch 3/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 418ms/step - accuracy: 0.0705 - loss: 3.2810 - val_accuracy: 0.0884 - val_loss: 3.2554
Epoch 4/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 406ms/step - accuracy: 0.0634 - loss: 3.2877 - val_accuracy: 0.0846 - val_loss: 3.2539
Epoch 5/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 408ms/step - accuracy: 0.0729 - loss: 3.2649 - val_accuracy: 0.0840 - val_loss: 3.2481
Epoch 6/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 416ms/step - accuracy: 0.0700 - loss: 3.2602 - val_accuracy: 0.0833 - val_loss: 3.2434
Epoch 7/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 416ms/step - accuracy: 0.0726 - loss: 3.2621 - val_accuracy: 0.0821 - val_loss: 3.2404
Epoch 8/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 410ms/step - accuracy: 0.0664 - loss: 3.2567 - 

In [ ]:
history2 = model.fit(
    X_train,
    y_train,
    validation_data=(X_test,y_test),
    epochs=300,
    batch_size=128
)

In [18]:
from tensorflow.keras import layers, models
import tensorflow.keras.applications

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    input_shape=(224,224,3),
    classes=31,
    classifier_activation='softmax',
)
eff_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(31)
])

eff_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history_eff = eff_model.fit(    X_train,
    y_train,
    validation_data=(X_test,y_test),
    epochs=300,
    batch_size=64)

Epoch 1/300


E0000 00:00:1746730135.197757    5941 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1746730135.356253    5941 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1746730136.016472    5941 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1746730136.158813    5941 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1746730136.621732    5941 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

98/99 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.1119 - loss: 7.7599

E0000 00:00:1746730195.253879    5941 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1746730195.391610    5941 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


99/99 ━━━━━━━━━━━━━━━━━━━━ 171s 745ms/step - accuracy: 0.1127 - loss: 7.7289 - val_accuracy: 0.0344 - val_loss: 12.7860
Epoch 2/300
99/99 ━━━━━━━━━━━━━━━━━━━━ 27s 274ms/step - accuracy: 0.3115 - loss: 3.7862 - val_accuracy: 0.0344 - val_loss: 14.1246
Epoch 3/300
99/99 ━━━━━━━━━━━━━━━━━━━━ 26s 265ms/step - accuracy: 0.3052 - loss: 4.0444 - val_accuracy: 0.0121 - val_loss: 3.4578
Epoch 4/300
99/99 ━━━━━━━━━━━━━━━━━━━━ 26s 257ms/step - accuracy: 0.0844 - loss: 7.3891 - val_accuracy: 0.0331 - val_loss: 14.0836
Epoch 5/300
99/99 ━━━━━━━━━━━━━━━━━━━━ 25s 252ms/step - accuracy: 0.0315 - loss: 3.4959 - val_accuracy: 0.0413 - val_loss: 14.7362
Epoch 6/300
99/99 ━━━━━━━━━━━━━━━━━━━━ 25s 251ms/step - accuracy: 0.0338 - loss: 3.4639 - val_accuracy: 0.0407 - val_loss: 8.6129
Epoch 7/300
99/99 ━━━━━━━━━━━━━━━━━━━━ 25s 251ms/step - accuracy: 0.0324 - loss: 3.4406 - val_accuracy: 0.0401 - val_loss: 3.4520
Epoch 8/300
99/99 ━━━━━━━━━━━━━━━━━━━━ 25s 249ms/step - accuracy: 0.0343 - loss: 3.4432 - val_acc

KeyboardInterrupt: 

In [ ]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
def remove_corrupt_images(directory):
    invalid = []
    for root, _, files in os.walk(directory):
        for f in files:
            if f.lower().endswith('.jpg') or f.lower().endswith('.jpeg'):
                path = os.path.join(root, f)
                try:
                    with Image.open(path) as img:
                        img.verify()
                except Exception as e:
                    invalid.append(path)
    return invalid

invalid = remove_corrupt_images("/kaggle/input/rgb-arabic-alphabets-sign-language-dataset/RGB ArSL dataset")

In [ ]:
len(invalid)

In [1]:
import os
import cv2
import torch
from torch.utils.data import Dataset
from transformers import AutoImageProcessor

processor = AutoImageProcessor.from_pretrained("google/siglip-base-patch16-224",use_fast = True)

class ArSLDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.samples = []
        self.labels_map = {}
        self.processor = processor

        label_id = 0
        for dirname, _, filenames in os.walk(root_dir):
            label = os.path.basename(dirname)
            if not filenames or label.startswith("."):
                continue  # skip hidden or empty folders
            if label not in self.labels_map:
                self.labels_map[label] = label_id
                label_id += 1
            for filename in filenames:
                filepath = os.path.join(dirname, filename)
                self.samples.append((filepath, self.labels_map[label]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        try:
            # Use cv2 to read the image (convert to RGB)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except:
            print(f"Failed to load image: {img_path}")
            image = 255 * np.ones((224, 224, 3), dtype=np.uint8)  # fallback: white image

        encoding = self.processor(images=image, return_tensors="pt")
        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }


root = "/kaggle/input/rgb-arabic-alphabets-sign-language-dataset/RGB ArSL dataset"
dataset = ArSLDataset(root, processor)


from torch.utils.data import random_split


train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


2025-05-09 11:15:59.028771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746789359.263555      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746789359.344609      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

In [2]:
from transformers import AutoImageProcessor, ResNetForImageClassification

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")

res_model = ResNetForImageClassification.from_pretrained(
    "microsoft/resnet-50",
    num_labels=31,ignore_mismatched_sizes=True
)

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([31]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([31, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainingArguments
from transformers import DefaultDataCollator
from sklearn.metrics import accuracy_score
data_collator = DefaultDataCollator()

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}
    
training_args = TrainingArguments(
    output_dir="./results_resnet",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
     weight_decay=0.01,
    eval_strategy ="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    report_to="none",  
    load_best_model_at_end=True,
    dataloader_num_workers=4,   
    dataloader_pin_memory=True
)
    
from transformers import Trainer

trainer = Trainer(
    model=res_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    processing_class = processor,
    compute_metrics=compute_metrics
)


In [17]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,3.325900,3.292609,0.154580
2,3.163600,3.030396,0.306616
3,2.883400,2.719919,0.468193
4,2.625900,2.525049,0.564249
5,2.478000,2.445314,0.575064


Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warning

TrainOutput(global_step=495, training_loss=2.8953519878965435, metrics={'train_runtime': 2473.2591, 'train_samples_per_second': 12.704, 'train_steps_per_second': 0.2, 'total_flos': 6.689025974215066e+17, 'train_loss': 2.8953519878965435, 'epoch': 5.0})

In [18]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,2.121600,1.722301,0.700382
2,1.527100,1.217066,0.795802
3,1.157600,0.960307,0.842239
4,0.950900,0.846429,0.865776
5,0.858300,0.805682,0.869593


Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warning

TrainOutput(global_step=495, training_loss=1.323099849200008, metrics={'train_runtime': 2428.8186, 'train_samples_per_second': 12.936, 'train_steps_per_second': 0.204, 'total_flos': 6.689025974215066e+17, 'train_loss': 1.323099849200008, 'epoch': 5.0})

In [19]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.704200,0.541735,0.913486
2,0.488400,0.410288,0.925573
3,0.382100,0.341616,0.928117
4,0.314100,0.317125,0.936387
5,0.285700,0.297231,0.940204


Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warning

TrainOutput(global_step=495, training_loss=0.43491293011289656, metrics={'train_runtime': 2505.7328, 'train_samples_per_second': 12.539, 'train_steps_per_second': 0.198, 'total_flos': 6.689025974215066e+17, 'train_loss': 0.43491293011289656, 'epoch': 5.0})

In [22]:
metrics = trainer.evaluate()

Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [23]:
metrics

{'eval_loss': 0.29723137617111206,
 'eval_accuracy': 0.9402035623409669,
 'eval_runtime': 92.7216,
 'eval_samples_per_second': 16.954,
 'eval_steps_per_second': 0.27,
 'epoch': 5.0}

In [24]:
trainer.save_model("resnet50-final model")

In [25]:
import shutil

# Define source folder and output zip path (no need to add .zip to output name)
source_folder = '/kaggle/working/resnet50-final model'
output_zip = '/kaggle/working/resnet50-94'  # This will create output.zip

# Create the zip archive
shutil.make_archive(output_zip, 'zip', source_folder)

'/kaggle/working/resnet50-94.zip'